In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

In [2]:
! rm -r sketch2ppt
! git clone https://github.com/fateh288/sketch2ppt.git



Cloning into 'sketch2ppt'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 96 (delta 43), reused 76 (delta 28), pack-reused 0
Unpacking objects: 100% (96/96), done.


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
import h5py
def load_data():
    f = h5py.File('sketch2ppt/hdf5/shapes_classification.hdf5', 'r')
    return (f['train_img'][...],f['train_labels'][...]),(f['test_img'][...],f['test_labels'][...])

In [5]:
(train_images,train_labels),(test_images,test_labels) = load_data()
#train_images, test_images = train_images / 255.0, test_images / 255.0
print(train_images.shape, type(train_images))
print(train_labels.shape, type(train_labels))

(200, 64, 64, 1) <class 'numpy.ndarray'>
(200,) <class 'numpy.ndarray'>


In [10]:
IMG_SIZE = 64

data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.8),
])
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255)
])

model = models.Sequential()
model.add(resize_and_rescale)
# model.add(data_augmentation)
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(5, activation='softmax'))

In [11]:
custom_adam = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=custom_adam,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=2000,
                    validation_data=(test_images, test_labels))


Epoch 1/2000
7/7 [==============================] - 1s 97ms/step - loss: 1.6098 - accuracy: 0.2550 - val_loss: 1.6096 - val_accuracy: 0.1600
Epoch 2/2000
7/7 [==============================] - 0s 69ms/step - loss: 1.6104 - accuracy: 0.2250 - val_loss: 1.6097 - val_accuracy: 0.1600
Epoch 3/2000
7/7 [==============================] - 0s 69ms/step - loss: 1.6094 - accuracy: 0.2050 - val_loss: 1.6095 - val_accuracy: 0.1600
Epoch 4/2000
7/7 [==============================] - 0s 68ms/step - loss: 1.6092 - accuracy: 0.2150 - val_loss: 1.6095 - val_accuracy: 0.1600
Epoch 5/2000
7/7 [==============================] - 0s 68ms/step - loss: 1.6095 - accuracy: 0.2000 - val_loss: 1.6095 - val_accuracy: 0.2000
Epoch 6/2000
7/7 [==============================] - 0s 69ms/step - loss: 1.6095 - accuracy: 0.1900 - val_loss: 1.6095 - val_accuracy: 0.1800
Epoch 7/2000
7/7 [==============================] - 0s 69ms/step - loss: 1.6077 - accuracy: 0.1950 - val_loss: 1.6097 - val_accuracy: 0.1800
Epoch 8/2000


In [12]:
model.save('/content/gdrive/My Drive/saved_model_sketch2ppt/model_conv_fully_drop2000')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/gdrive/My Drive/saved_model_sketch2ppt/model_conv_fully_drop2000/assets


In [ ]:
loaded_model = tf.keras.models.load_model('/content/gdrive/My Drive/saved_model_sketch2ppt/model_fully_connected')
loaded_model.summary()



Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_1 (Sequential)    (None, 64, 64, 1)         0         
_________________________________________________________________
flatten (Flatten)            (None, 4096)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               524416    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_4 (Dense)              (None, 5)                